In [1]:
import pandas as pd
import scipy.stats as st
import math
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [38]:
# ! pip install pyvttbl

In [14]:
ex_data = pd.read_excel('hw3.xlsx', sheet_name="ex_8.6")
ex_data.set_index('Device', inplace=True)
# ex_data = ex_data.T
ex_data

,1,2,3,4,5,6
Device,,,,,,
A,-0.307,-0.294,0.079,0.019,-0.136,-0.324
B,-0.176,0.125,-0.013,0.082,0.091,0.459
C,0.137,-0.063,0.240,-0.050,0.318,0.154
D,-0.042,0.690,0.201,0.166,0.219,0.407


In [16]:
st.f_oneway(ex_data.loc["A"], ex_data.loc["B"], ex_data.loc["C"], ex_data.loc["D"])

F_onewayResult(statistic=4.847023848816111, pvalue=0.010773308873392276)

In [11]:
pwine_data = pd.read_excel('hw3.xlsx', sheet_name="port_wine")
pwine_data.set_index('Patient', inplace=True)
pwine_data

,0–5,6–11,12–17,18–31
Patient,,,,
1,9.6938,13.4081,10.9110,1.4352
2,7.0027,8.2520,10.3844,10.7740
3,10.3249,12.0098,6.4080,8.4292
4,2.7491,7.4514,13.5611,4.4898
5,0.5637,6.9131,3.4523,13.6303
6,8.0739,5.6594,9.5427,4.1640
7,0.1440,8.7352,10.4976,5.4684
8,8.4572,0.2510,4.6775,4.8650
9,2.0162,8.9991,24.7156,3.0733


In [72]:
pwine_melt = pwine_data.melt()

In [73]:
residuals = pwine_data - pwine_data.mean()
# residuals_t = residuals.T

In [74]:
pg.anova(data=pwine_melt, dv = 'value', between='variable', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,variable,107.984638,3,35.994879,1.945663,0.128362,0.064258
1,Within,1572.505089,85,18.500060,NaN,NaN,NaN


In [75]:
# get first 15 rows of residuals and square them
res_sq_table = residuals[0:15]**2

# sum all residuals
ssw = res_sq_table.values.sum()
ssw

1145.0972685492352

In [76]:
# expected MSE = 18.5
ssw / 12

95.42477237910293

In [60]:
# r_melt = residuals.melt()
# r_melt

In [14]:
# pwine_data.iloc[0, 1] - 7.223546

In [15]:
# pwine_data.mean()

### ANOVA library exploration

In [45]:
mod = ols('value ~ variable', data=pwine_melt).fit()
sm.stats.anova_lm(mod, typ=2)

,sum_sq,df,F,PR(>F)
variable,107.984638,3.0,1.945663,0.128362
Residual,1572.505089,85.0,NaN,NaN


In [32]:
pg.pairwise_tukey(data=pwine_melt, dv='value', between='variable')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,0–5,12–17,4.998790,7.757281,-2.758490,1.327370,-2.078163,0.168590,-0.629234
1,0–5,18–31,4.998790,5.682191,-0.683401,1.298193,-0.526425,0.952467,-0.156033
2,0–5,6–11,4.998790,7.223546,-2.224755,1.285220,-1.731030,0.314197,-0.508170
3,12–17,18–31,7.757281,5.682191,2.075090,1.298193,1.598444,0.385007,0.473781
4,12–17,6–11,7.757281,7.223546,0.533735,1.285220,0.415287,0.975717,0.121914
5,18–31,6–11,5.682191,7.223546,-1.541355,1.255064,-1.228108,0.610858,-0.352351
